In [1]:
!wget https://setup.johnsnowlabs.com/colab.sh -O - | bash /dev/stdin -p 3.4.1 -s 5.1.2 -g

--2023-12-10 19:06:49--  https://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-12-10 19:06:49--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-12-10 19:06:49 (95.1 MB/s) - written to stdout [1191/1191]

Installi

In [2]:
import pandas as pd

In [3]:
file_path = 'annotated-titles.csv'
df = pd.read_csv(file_path,encoding='latin1')

In [4]:
df.head(3)

,title,label,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,To require the Secretary of Agriculture to can...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,To require the prompt reconstruction of and re...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Expressing support for the designation of Sept...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
unique_labels = df['label'].unique()
print(unique_labels)
label_counts = df['label'].value_counts()
print(label_counts)

['-' '1' '0' nan '#VALUE!' '-1' ' and for other purposes."']
-                            232487
0                             23262
1                               667
-1                               76
 and for other purposes."         2
#VALUE!                           1
Name: label, dtype: int64


In [6]:
allowed_values = ['-', '0', '1']
df = df[df['label'].isin(allowed_values)]

In [7]:
selected_columns = ['title', 'label']
df = df[selected_columns]

In [8]:
df['label'] = df['label'].replace('-', '0')
label_counts = df['label'].value_counts()
print(label_counts)
df.head()

0    255749
1       667
Name: label, dtype: int64


,title,label
0,To require the Secretary of Agriculture to can...,0
1,To require the prompt reconstruction of and re...,0
2,Expressing support for the designation of Sept...,0
3,To amend the State Department Basic Authoritie...,0
4,Electing Members to the Joint Committee of Con...,0


In [9]:
df.to_csv('data.csv', index=False)

In [10]:
import os
import sys

import sparknlp

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import pandas as pd

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.1.2
Apache Spark version:  3.2.3


In [11]:
dataDF = spark.read \
      .option("header", True) \
      .csv("data.csv")

dataDF.show(truncate=50)

+--------------------------------------------------+-----+
|                                             title|label|
+--------------------------------------------------+-----+
|To require the Secretary of Agriculture to canc...|    0|
|To require the prompt reconstruction of and res...|    0|
|Expressing support for the designation of Septe...|    0|
|To amend the State Department Basic Authorities...|    0|
|Electing Members to the Joint Committee of Cong...|    0|
|To authorize the Bureau of Reclamation to parti...|    0|
|To reauthorize and amend title XIV of the Publi...|    0|
|To amend the National Wool Act of 1954 to reduc...|    0|
|To amend the Act establishing Golden Gate Natio...|    0|
|An act to amend the National Wool Act of 1954 t...|    0|
|An act to amend the National Wool Act of 1954 t...|    0|
|To provide for the use of Department of Defense...|    0|
|To make systematic and comprehensive reductions...|    0|
|To amend the National Labor Relations Act and t...|    

In [34]:
from pyspark.sql.functions import col, explode
from pyspark.ml.feature import Tokenizer, StopWordsRemover


label1_df = dataDF.filter(col("label") == 1)

tokenizer = Tokenizer(inputCol="title", outputCol="words")
tokenized_df = tokenizer.transform(label1_df)

stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_df = stopwords_remover.transform(tokenized_df)

word_counts = (filtered_df.select(explode(col("filtered_words")).alias("word"))
               .groupBy("word")
               .count()
               .orderBy(col("count").desc()))

word_counts.show(200)

+---------------+-----+
|           word|count|
+---------------+-----+
|      purposes.|  373|
| transportation|  252|
|          amend|  244|
|         states|  232|
|         united|  222|
|            act|  211|
|       security|  159|
|        highway|  153|
|          title|  138|
|          code,|  137|
|      secretary|  130|
|        provide|  128|
|          motor|  126|
|        certain|   91|
|        vehicle|   83|
|            49,|   78|
|         direct|   75|
|        safety,|   74|
|       vehicles|   71|
|        require|   69|
|        privacy|   65|
|      authorize|   63|
|      establish|   61|
|         safety|   61|
|        federal|   59|
|       national|   59|
|      providing|   53|
|          state|   51|
|        related|   50|
|transportation,|   49|
|        improve|   49|
|          (h.r.|   48|
|           bill|   48|
|       programs|   47|
|       homeland|   47|
|  consideration|   45|
|           year|   43|
|    federal-aid|   42|
|       prohibit

In [12]:
from pyspark.sql.functions import col

dataDF.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+------+
|               label| count|
+--------------------+------+
|                   0|255725|
|                   1|   667|
| 166 degrees 34'1...|     5|
|       1789-1993""."|     4|
|      166 degrees 34|     1|
| and for other pu...|     1|
|             (4""r)-|     1|
|              Design|     1|
|       1853-1861""."|     1|
|       1917-1990""."|     1|
| The Committee on...|     1|
| sponsored by the...|     1|
| launched by Pres...|     1|
|       1870-1989""."|     1|
|5-pentanediyl) bi...|     1|
| as a party to th...|     1|
| and for other pu...|     1|
| the publication ...|     1|
|2-ethane-diyl-bis...|     1|
+--------------------+------+



In [13]:
dataDF = dataDF.withColumn("label", dataDF["label"].cast("int"))

filtered_dataDF = dataDF.filter((col("label") == "0") | (col("label") == "1"))

filtered_dataDF.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-----+------+
|label| count|
+-----+------+
|    0|255725|
|    1|   667|
+-----+------+



In [14]:
useEmbeddings = UniversalSentenceEncoder.pretrained()\
      .setInputCols("document")\
      .setOutputCol("use_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [15]:
from pyspark.ml.feature import StringIndexer
document_assembler = DocumentAssembler() \
      .setInputCol("title") \
      .setOutputCol("document")

loaded_useEmbeddings = UniversalSentenceEncoder.load('/root/cache_pretrained/tfhub_use_en_2.4.0_2.4_1587136330099')\
      .setInputCols("document")\
      .setOutputCol("use_embeddings")

embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["use_embeddings"]) \
      .setOutputCols(["finished_use_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

use_pipeline = Pipeline(
      stages=[
        document_assembler,
        loaded_useEmbeddings,
        embeddings_finisher,
        ]
      )

use_df = use_pipeline.fit(filtered_dataDF).transform(filtered_dataDF)

In [16]:
use_df.show(2)

+--------------------+-----+--------------------+--------------------+-----------------------+
|               title|label|            document|      use_embeddings|finished_use_embeddings|
+--------------------+-----+--------------------+--------------------+-----------------------+
|To require the Se...|    0|[{document, 0, 75...|[{sentence_embedd...|   [[-0.007805520202...|
|To require the pr...|    0|[{document, 0, 12...|[{sentence_embedd...|   [[-0.022778689861...|
+--------------------+-----+--------------------+--------------------+-----------------------+
only showing top 2 rows



In [17]:
from pyspark.sql.functions import explode
use_df= use_df.withColumn("features", explode(use_df.finished_use_embeddings))
use_df.show(2)

+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
|               title|label|            document|      use_embeddings|finished_use_embeddings|            features|
+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
|To require the Se...|    0|[{document, 0, 75...|[{sentence_embedd...|   [[-0.007805520202...|[-0.0078055202029...|
|To require the pr...|    0|[{document, 0, 12...|[{sentence_embedd...|   [[-0.022778689861...|[-0.0227786898612...|
+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
only showing top 2 rows



In [18]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# array_to_vector = udf(lambda arr: Vectors.dense(arr), VectorUDT())
# use_df = use_df.withColumn("vector_col", array_to_vector(use_df["features"]))


In [19]:
from pyspark.sql.functions import col

num_label_1 = 667
num_label_0 = 257513

label_1_df = use_df.filter(col("label") == 1)

sampled_label_1_df = use_df.sample(False, num_label_1 / label_1_df.count())

label_0_df = use_df.filter(col("label") == 0)

sampled_label_0_df = label_0_df.sample(False, num_label_0 / label_0_df.count())

final_sampled_df = sampled_label_0_df.union(sampled_label_1_df)

final_sampled_df.groupBy("label") \
    .count() \
    .show()


+-----+------+
|label| count|
+-----+------+
|    0|257513|
|    1|   667|
+-----+------+



In [20]:
final_sampled_df.show(2)

+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
|               title|label|            document|      use_embeddings|finished_use_embeddings|            features|
+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
|To require commer...|    0|[{document, 0, 13...|[{sentence_embedd...|   [[-0.045169189572...|[-0.0451691895723...|
|To amend the Noni...|    0|[{document, 0, 16...|[{sentence_embedd...|   [[-0.053431000560...|[-0.0534310005605...|
+--------------------+-----+--------------------+--------------------+-----------------------+--------------------+
only showing top 2 rows



In [21]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scalerModel = scaler.fit(final_sampled_df)

final_data_output = scalerModel.transform(final_sampled_df)

final_data_output.select("title", "scaledFeatures").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
from pyspark.ml.clustering import KMeans
KMeans1=KMeans(featuresCol='scaledFeatures', k=2)
KMeans_fit=KMeans1.fit(final_data_output)


In [23]:
# Shows the result.
centers = KMeans_fit.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[-8.41972691e-02 -1.62754085e-01 -6.78215868e-01  5.19577251e-02
 -6.96056574e-01 -3.79242228e-01  1.82555821e+00 -4.00439716e-02
 -1.27493293e+00  1.11249238e-01  1.69618218e-01  8.28776666e-02
 -2.73017391e-01 -6.34938361e-01 -7.55323837e-01 -2.52799372e-01
 -2.33649889e-02  1.55675160e-01  1.81909857e+00 -2.27322804e-01
  7.92079441e-01 -1.85237205e+00 -4.17043080e-01  3.07950428e+00
  3.74652535e-01  4.80155702e-01  1.00702022e-01 -1.58595883e+00
 -4.45030297e-01 -4.08606080e-01 -8.39150254e-01 -5.97436198e-01
 -6.22405250e-01  3.32356101e-02 -1.01915335e+00  6.27337303e-01
 -4.60998161e-01  4.05012841e-01 -2.06452164e-01 -1.05444461e+00
  2.62490914e-01 -4.13467500e-01  3.41039075e-01  2.20155523e+00
  2.67069433e-01  2.65018564e+00 -6.97247769e-01 -6.79346859e-02
 -7.99263719e-01 -2.16698501e+00  3.93500547e-01 -1.53655766e+00
 -1.56365711e-01  2.16385997e+00  1.52377309e+00  4.58707464e+00
  1.69018460e+00 -1.20834835e+00 -4.40184481e-01 -5.87601724e-01
  5.188

In [24]:
KMeans_fit.transform(final_data_output).groupBy('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|         1|192474|
|         0| 65706|
+----------+------+



In [25]:
result = KMeans_fit.transform(final_data_output).select('prediction')

In [26]:
new_result = dataDF.join(result)

In [30]:
new_result.show(2)

+--------------------+-----+----------+
|               title|label|prediction|
+--------------------+-----+----------+
|To require the Se...|    0|         1|
|To require the Se...|    0|         1|
+--------------------+-----+----------+
only showing top 2 rows



In [32]:
new_result.write.csv("results.csv", header=True)